In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
import cv2
import imutils
from skimage import exposure
B= []
for i in range(len(mnist_train_x)):
    A = mnist_train_x[i]
    A = exposure.rescale_intensity(A, out_range=(0, 255))
    A = imutils.resize(A, width=32)
    B.append(A)
B = np.array(B)

mnist_train_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)
B= []
for i in range(len(mnist_test_x)):
    A = mnist_test_x[i]
    A = exposure.rescale_intensity(A, out_range=(0, 255))
    A = imutils.resize(A, width=32)
    B.append(A)
B = np.array(B)

mnist_test_RGB_x = np.repeat(B[:,:, :, np.newaxis], 3, axis=3)

Using TensorFlow backend.


In [2]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:04_Central_Daylight_Time_2018
Cuda compilation tools, release 10.0, V10.0.130


In [3]:
M_train_y = np.array([[mnist_train_y[i]] for i in range(len(mnist_train_y))])
M_test_y = np.array([[mnist_test_y[i]] for i in range(len(mnist_test_y))])
(C_x_train, C_y_train), (C_x_test, C_y_test) = cifar10.load_data()

In [4]:
class MNIST_vgg:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('MNIST_vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 25
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        x_train,y_train,x_test,y_test = mnist_train_RGB_x,M_train_y,mnist_test_RGB_x,M_test_y
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('MNIST_vgg.h5')
        return model

if __name__ == '__main__':


    x_train,y_train,x_test,y_test = mnist_train_RGB_x,M_train_y,mnist_test_RGB_x,M_test_y
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = MNIST_vgg()

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

W1001 19:46:33.125200 25180 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 19:46:33.152634 25180 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1001 19:46:33.158618 25180 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1001 19:46:33.186544 25180 deprecation_wrapper.py:119] From c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprec

Epoch 1/25
 - 50s - loss: 20.0786 - acc: 0.5210 - val_loss: 21.2302 - val_acc: 0.1212
Epoch 2/25
 - 44s - loss: 8.4775 - acc: 0.8307 - val_loss: 8.0610 - val_acc: 0.3570
Epoch 3/25
 - 43s - loss: 3.6573 - acc: 0.9149 - val_loss: 5.3086 - val_acc: 0.3078
Epoch 4/25
 - 44s - loss: 1.7818 - acc: 0.9394 - val_loss: 1.5397 - val_acc: 0.8567
Epoch 5/25
 - 43s - loss: 1.0077 - acc: 0.9495 - val_loss: 1.2960 - val_acc: 0.7996
Epoch 6/25
 - 43s - loss: 0.7414 - acc: 0.9516 - val_loss: 1.6076 - val_acc: 0.8033
Epoch 7/25
 - 43s - loss: 0.6663 - acc: 0.9553 - val_loss: 0.5313 - val_acc: 0.9734
Epoch 8/25
 - 43s - loss: 0.5612 - acc: 0.9574 - val_loss: 0.5151 - val_acc: 0.9616
Epoch 9/25
 - 43s - loss: 0.5314 - acc: 0.9593 - val_loss: 0.4770 - val_acc: 0.9696
Epoch 10/25
 - 43s - loss: 0.5040 - acc: 0.9610 - val_loss: 0.5529 - val_acc: 0.9469
Epoch 11/25
 - 44s - loss: 0.4950 - acc: 0.9615 - val_loss: 0.4368 - val_acc: 0.9710
Epoch 12/25
 - 43s - loss: 0.4853 - acc: 0.9621 - val_loss: 0.4181 - val

In [5]:
import numpy as np
def get_submax(arr):
    arr = np.array(arr)
    MAX = np.max(arr)
    idx = find_idx(arr, MAX)
    arr_without_max = np.delete(arr,idx)
    return np.max(arr_without_max)
def find_statistics(Prob_Mat):
    Prob_diff = []
    MAX_Prob_Mat = []
    MAX_Prob_Mat_idx = []
    subMAX_Prob_Mat = []
    subMAX_Prob_Mat_idx = []
    for i in range(len(Prob_Mat)):
        MAX = np.max(Prob_Mat[i])
        MAX_idx = find_idx(Prob_Mat[i], MAX)[0]
        subMAX = get_submax(Prob_Mat[i])
        subMAX_idx = find_idx(Prob_Mat[i], subMAX)[0]
        prob_difference = MAX - subMAX
        Prob_diff.append(prob_difference)
        MAX_Prob_Mat.append(MAX)
        subMAX_Prob_Mat.append(subMAX)
        MAX_Prob_Mat_idx.append(MAX_idx)
        subMAX_Prob_Mat_idx.append(subMAX_idx)
    return Prob_diff,MAX_Prob_Mat,MAX_Prob_Mat_idx,subMAX_Prob_Mat,subMAX_Prob_Mat_idx
import pandas as pd
def entropy_from_distribution(p, axis):
    return np.log(10.) + np.sum(p * np.log(np.abs(p) + 1e-11), axis=1, keepdims=True)
def conclusion_df(statistics, entropy):
    d = {'Prob_diff': statistics[0], 'MAX_Prob_Mat': statistics[1], 'MAX_Prob_Mat_idx': statistics[2], 'subMAX_Prob_Mat': statistics[3], 'subMAX_Prob_Mat_idx': statistics[4], 'entropy':entropy}
    df = pd.DataFrame(data=d)
    return df
def plot_avg_dist(Prob_Mat):
    plt.figure(figsize=(18, 12))
    for i in range(10):
        plt.subplot(2, 5, i+1)
        plt.hist(Prob_Mat[:,i], bins=20)
        plt.title('Prob_dist' + str(i))
    plt.show()
    return
def find_idx(arr, target):
    ans = []
    for i in range(len(arr)):
        if arr[i] == target:
            ans.append(i)
    return ans
import matplotlib.pyplot as plt


In [6]:
def report_ID_OD(predicted_ID, predicted_OD):

    statistics_ID = find_statistics(predicted_ID)
    statistics_OD = find_statistics(predicted_OD)
    entropy_ID = entropy_from_distribution(predicted_ID, axis=1)
    entropy_ID = entropy_from_distribution(predicted_ID, axis=1).reshape(entropy_ID.shape[0])

    entropy_OD = entropy_from_distribution(predicted_OD, axis=1)
    entropy_OD = entropy_from_distribution(predicted_OD, axis=1).reshape(entropy_OD.shape[0])
    df_ID_ID, df_ID_OD = conclusion_df(statistics_ID, entropy_ID), conclusion_df(statistics_OD, entropy_OD)

    plot_avg_dist(predicted_ID)
    plot_avg_dist(predicted_OD)
    plt.figure(figsize=(18, 12))
    plt.subplot(2, 3, 1)
    plt.hist(df_ID_ID['Prob_diff'], bins=20)
    plt.title('ID_ID_prob_diff')
    plt.subplot(2, 3, 2)
    plt.hist(df_ID_ID['MAX_Prob_Mat_idx'], bins=20)
    plt.title('ID_ID_max')
    plt.subplot(2, 3, 3)
    plt.hist(df_ID_ID['entropy'], bins=20)
    plt.title('ID_ID_entropy')
    plt.subplot(2, 3, 4)
    plt.hist(df_ID_OD['Prob_diff'], bins=20)
    plt.title('ID_OD_prob_diff')
    plt.subplot(2, 3, 5)
    plt.hist(df_ID_OD['MAX_Prob_Mat_idx'], bins=20)
    plt.title('ID_OD_max')
    plt.subplot(2, 3, 6)
    plt.hist(df_ID_OD['entropy'], bins=20)
    plt.title('ID_OD_entropy')
    plt.show()
